# Implementation of Open Domain Q.A. system for Covid19

In this assignment you will build an application that performs __information retrieval__ and __question answering__, which are the core tasks that take care the **Open Domain Q.A.** task. 

*   __Information Retrieval__ Retrieve documents with information that is relevant to the user’s information need and helps the user complete a task.

* __Question Answering__ Systems that automatically answer questions posed by humans in a natural language.

The steps are the following:
- Preprocess the data and init models for I.R.
- Preprocess the data and init models for Q.A.
- Implement the functions: retrieve_documents & answer question given a query.
- Run some analysis of the model output.

Note you are not requiered to strictly follow the steps suggested in the notebook as long as you build a model and provide some analysis.


In [ ]:
#TODO Use this snippet code as a guide, and feel free to modify it as you want.

class IR(object):
    def __init__():
        pass

    def retrieve_documents(query,topk):
        pass

class QA(object):
    def __init__():
        pass

    def answer_question(query,passage):
        pass

#TODO a pseudocode for the execution of the model

# query=''
# topk_docs=retrieve_documents(query)
# answer=answer_question(query,topk_docs[0])
# pritn(answer)

## Analysis of the output
It is important to run some analysis of the output of our model in order to gain some insights about good and bad things of the model. Use some questions from the lab 8.IR_for_covid19 to test your model.

In addition, each function should return a score. This will allow us to better understand how confident the model is returning both documents and answers. The scoring by default should be best retrieved document & best answer in that retrieved document.

### TO-DO 
- Build the Open Domain Q.A. system.
- Show some correct and incorrect examples for IR & QA for Covid 19 and its scores.
### TO-DO (optional)
- Try to improve over the presented baseline scoring system, for example, returning the highest scored answer in the first 10 most relevant documents, feel free to explore any other scoring strategy.
- Apply a threshold based on the score to filter out unwanted results (low scored answers).